In [9]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import joblib

In [21]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlfow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///c:/Users/wzhiy/OneDrive/Desktop/MLops/mlops-zoomcamp/02-Experiment_Tracking/mlruns/1', creation_time=1748169424951, experiment_id='1', last_update_time=1748169424951, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

# Train the model

In [5]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")
df_val = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")

In [6]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.duration.describe()
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

df[categorical] = df[categorical].astype(str)

In [8]:
train_dict = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
target = 'duration'
y = df[target].values

In [14]:
with mlflow.start_run():
    mlflow.set_tag("developer", "wen")
    mlflow.log_param("train-data-path", "../data/yellow_tripdata_2023-01.parquet")
    mlflow.log_param("valid-data-path", "../data/yellow_tripdata_2023-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y)

    y_pred = lr.predict(X_train)
    rmse = mean_squared_error(y, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)        

In [11]:
lr = LinearRegression()
lr.fit(X_train, y)
joblib.dump(lr, 'linear_regression_model.joblib')

['linear_regression_model.joblib']

In [12]:
y_pred = lr.predict(X_train)

In [13]:
mean_squared_error(y, y_pred, squared=False)

7.649261954021325

# Run val set

In [ ]:
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib

In [20]:
lr = joblib.load('./linear_regression_model.joblib')

In [15]:
df_2 = pd.read_parquet("../data/yellow_tripdata_2023-02.parquet")

In [ ]:

df_2['tpep_pickup_datetime'] = pd.to_datetime(df_2['tpep_pickup_datetime'])
df_2['tpep_dropoff_datetime'] = pd.to_datetime(df_2['tpep_dropoff_datetime'])
df_2['duration'] = (df_2['tpep_dropoff_datetime'] - df_2['tpep_pickup_datetime']).dt.total_seconds() / 60
df_2 = df_2[(df_2.duration >= 1) & (df_2.duration <= 60)].copy()
df_2['PULocationID'] = df_2['PULocationID'].astype(str)
df_2['DOLocationID'] = df_2['DOLocationID'].astype(str)
val_dicts = df_2[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df_2[target].values

In [22]:
X_val.shape

(2855951, 515)

In [23]:
y_pred_val = lr.predict(X_val)

In [24]:
mean_squared_error(y_val, y_pred_val, squared=False)

7.8118182188528005